In [8]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

prompt_template = "Generate a concise story idea for a {genre} book."

llm = ChatOpenAI(temperature=0.7)
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt_template))
result = llm_chain("sci-fi")
print(result)

{'genre': 'sci-fi', 'text': "In a future where the world has been ravaged by environmental disasters, a brilliant scientist discovers a way to manipulate time. With the ability to travel back and forth through different eras, she aims to prevent humanity's impending doom by altering key events in history. However, as she delves deeper into the past, she realizes that altering the course of history comes with unforeseen consequences, and she must confront the moral dilemma of whether to sacrifice the present for a better future."}


In [9]:
result["text"]

"In a future where the world has been ravaged by environmental disasters, a brilliant scientist discovers a way to manipulate time. With the ability to travel back and forth through different eras, she aims to prevent humanity's impending doom by altering key events in history. However, as she delves deeper into the past, she realizes that altering the course of history comes with unforeseen consequences, and she must confront the moral dilemma of whether to sacrifice the present for a better future."

---


In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
)

In [11]:
human_message_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        template="""I want you to brainstorm ideas for characters for my story. The genre is {genre}.
        Rank the ideas from most important to least important.
        
        Example response:
        1. Idea 1 ...,
        2. Idea 2 ...,
        3. Idea 3 ...
        
        """,
        input_variables=["genre"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9) 
chain = LLMChain(llm=chat, prompt=chat_prompt_template)

chain.invoke({"genre": "fantasy"})
# chain("fantasy") # This will work too using the __call__ method

{'genre': 'fantasy',
 'text': "1. The Protagonist: A young orphan with a mysterious past who discovers they have magical powers. They embark on a journey to uncover their true identity and save the world from an evil sorcerer.\n2. The Mentor: A wise and powerful wizard who guides the protagonist in their magical training. They provide guidance, teach important lessons, and share ancient knowledge about the world of magic.\n3. The Antagonist: The evil sorcerer who seeks to gain ultimate power by harnessing dark magic. They are ruthless, cunning, and will stop at nothing to achieve their goals. Their backstory and motivations are explored throughout the story.\n4. The Sidekick: A loyal and humorous companion to the protagonist, who provides comic relief and moral support. They may have their own unique abilities or skills that assist the protagonist on their journey.\n5. The Love Interest: A fellow magical being who becomes romantically involved with the protagonist. Their relationship a

In [12]:
genres = [
    {"genre": "fantasy"},
    {"genre": "sci-fi"},
]

# Generate multiple LLM requests based on multiple inputs:
# The async method is .aapply()
print(chain.apply(genres))

[{'text': "1. Protagonist - A young mage who discovers their true potential and becomes the key player in a battle against dark forces.\n2. Antagonist - A power-hungry sorcerer who seeks to gain ultimate control over the magical realm.\n3. Sidekick - A loyal and skilled warrior who aids the protagonist in their journey and provides valuable knowledge and support.\n4. Mentor - A wise old wizard who guides the protagonist and helps them develop their magical abilities.\n5. Love interest - A fellow mage who shares a deep connection with the protagonist and helps them navigate their emotions while balancing their responsibilities.\n6. Villain's henchmen - A group of skilled fighters or magical creatures who do the bidding of the antagonist and pose a constant threat to the protagonist.\n7. King/Queen - The ruler of the magical realm, who seeks to maintain peace and order but may have conflicting goals with the protagonist.\n8. Wise elder - A mystical being or wise creature who possesses kn

In [13]:
# The async method is .agenerate()
chain.generate(genres)

LLMResult(generations=[[ChatGeneration(text='1. A young, inexperienced wizard who discovers their magical abilities and must navigate the dangerous world of magic while trying to control their powers.\n2. A skilled thief and master of disguise who becomes entangled in a quest to retrieve a powerful artifact.\n3. A wise and powerful sorcerer who has been tasked with protecting a hidden city from dark forces.\n4. A skilled warrior from a nomadic tribe who is chosen by an ancient weapon to fulfill a prophecy and unite the warring factions of the realm.\n5. A mischievous and charismatic trickster who uses their wit and cunning to outsmart powerful enemies.\n6. A kind-hearted healer who possesses the ability to cure any illness or injury, but must contend with the moral dilemmas and responsibilities that come with such power.\n7. A mysterious and enigmatic figure who possesses ancient knowledge and serves as a guide and mentor to the main characters.\n8. A cursed creature who is both feared

Analyzing issues within a Chain object by its output alone can prove to be a daunting task, primarily because such objects often undergo substantial transformation—both before the input prompt is processed and after the LLM output is generated.

However, a solution lies within your reach. By setting the `verbose` parameter to `True`, you are enabling a feature that unveils the internal dynamics of the Chain object during its runtime. This provision can offer you invaluable insights, further empowering you to troubleshoot and resolve any underlying issues more effectively.


In [14]:
# You can debug all LLMChains by turning on Verbose=True:
chain = LLMChain(llm=chat, prompt=chat_prompt_template, verbose=True)
chain.invoke({"genre": "fantasy"})



> Entering new LLMChain chain...
Prompt after formatting:
System: I want you to brainstorm ideas for characters for my story. The genre is fantasy.
        Rank the ideas from most important to least important.
        
        Example response:
        1. Idea 1 ...,
        2. Idea 2 ...,
        3. Idea 3 ...
        
        

> Finished chain.


{'genre': 'fantasy',
 'text': "1. The Protagonist - A young orphan with a hidden magical ability who embarks on a journey to discover their true identity and save their world from an evil sorcerer.\n2. The Mentor - An ancient and wise magician who becomes the Protagonist's guide and teaches them about the magical realm and helps them develop their powers.\n3. The Villain - An enigmatic sorcerer who seeks to gain ultimate power by harnessing dark magic and plans to destroy the Protagonist and rule the world.\n4. The Sidekick - A loyal and brave companion who accompanies the Protagonist on their journey, providing comic relief and offering support in challenging situations.\n5. The Love Interest - A fellow magical being who captures the Protagonist's heart, but their relationship is complicated by their opposing loyalties or conflicting goals.\n6. The Rival - Another powerful wizard who is initially seen as an adversary but later becomes an unexpected ally or a source of constant competi

LLMChain does not automatically parse the output by default, even if the prompt object has an output parser. To apply the output parser on the LLM output you can add an output_parser.


In [15]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [16]:
class Ideas(BaseModel):
    ideas: List[str] = Field(description="A list of ideas for a story.")


parser = PydanticOutputParser(pydantic_object=Ideas)

prompt = PromptTemplate(
    template="I want you to brainstorm ideas for characters for my story. The genre is {genre}.\n  {format_instructions}",
    input_variables=["genre", "format_instructions"],
)

chain = LLMChain(llm=chat, prompt=prompt, output_parser=parser)

In [19]:
# Remember to use .run() instead of .invoke() when using output parsers:
result = chain.run(
    {"genre": "fantasy", "format_instructions": parser.get_format_instructions()}
)

In [21]:
print(result)   
print(result.dict())

ideas=['A young sorcerer with a mysterious past', 'A mischievous fairy who causes chaos wherever they go', 'A brave knight on a quest to save their kingdom', 'A wise old wizard with a penchant for riddles', 'A shape-shifting creature who can take on the form of any animal', 'A powerful witch who uses their magic for good', 'An eccentric inventor who creates fantastical gadgets', 'A cunning thief with a heart of gold', 'A noble princess with a secret identity as a skilled warrior', 'A group of unlikely allies who must band together to defeat a common enemy']
{'ideas': ['A young sorcerer with a mysterious past', 'A mischievous fairy who causes chaos wherever they go', 'A brave knight on a quest to save their kingdom', 'A wise old wizard with a penchant for riddles', 'A shape-shifting creature who can take on the form of any animal', 'A powerful witch who uses their magic for good', 'An eccentric inventor who creates fantastical gadgets', 'A cunning thief with a heart of gold', 'A noble p

/var/folders/_y/20jl658s4jl0zvy5c0x0c5140000gn/T/ipykernel_49364/1676782462.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  print(result.dict())


---


Given that your story generation will require multiple sequential prompts you can use a `SequentialChain` to chain multiple prompts together. The `SequentialChain` will automatically re-use the output of the previous prompt and use it as the input for the next prompt.


In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain

In [36]:
character_generation_prompt = PromptTemplate(
    template="""I want you to brainstorm 3 - 5 characters for my short story. The genre is {genre}.
    Each character must have a Name and a Biography.
    You must provide a name and biography for each character, this is very important!
    ---
    Example response:
    Name: CharWiz, Biography: A wizard who is a master of magic.
    Name: CharWar, Biography: A warrior who is a master of the sword.
    ---

    Characters: """,
    input_variables=["genre"],
)

plot_generation_prompt = PromptTemplate(
    template="""Given the following characters and the genre, create an effective plot for a short story:
    Characters:
    {characters} 
    ---
    Genre: {genre}
    ---
    Plot: """,
    input_variables=["genre", "characters"],
)

scene_generation_plot = PromptTemplate(
    template="""Act as an effective content creator. 
    Given multiple characters and a plot you are responsible generating the various scenes for each act. 
    
    You must de-compose the plot into multiple effective scenes:

    ---
    Characters:
    {characters}
    ---
    Genre: {genre}
    ---
    Plot: {plot}
    ---
    Example response:
    Scenes:
    Scene 1: Some text here.
    Scene 2: Some text here.
    Scene 3: Some text here.
    ----
    Scenes:
    """,
    input_variables=["genre", "characters", "plot"],
)

In [37]:
chat_model = ChatOpenAI()
character_generation_chain = LLMChain(
    llm=chat, prompt=character_generation_prompt, output_key="characters"
)
plot_generation_chain = LLMChain(
    llm=chat, prompt=plot_generation_prompt, output_key="plot"
)
scene_generation_chain = LLMChain(
    llm=chat, prompt=scene_generation_plot, output_key="scenes"
)

In [38]:
story_chain = SequentialChain(
    chains=[
        character_generation_chain,
        plot_generation_chain,
        scene_generation_chain,
    ],
    input_variables=["genre"],
    output_variables=["characters", "plot", "scenes", "genre"],
)

In [39]:
story_result = story_chain({"genre": "fantasy"})

In [27]:
print(story_result)

{'genre': 'fantasy', 'format_instructions': '', 'characters': "Name: Elara Dawnshield\nBiography: Elara Dawnshield is a skilled elven archer from the enchanted forest of Eldoria. Raised by a tribe of wise sages, she possesses an innate connection with nature and an unmatched accuracy with her bow. Elara is known for her fierce loyalty and unwavering determination to protect her homeland from any threat that may arise.\n\nName: Garrick Stormbringer\nBiography: Garrick Stormbringer is a mysterious and brooding sorcerer hailing from the remote mountains of Shadowpeak. He is rumored to have made a pact with an ancient elemental being, granting him unparalleled control over the forces of lightning and thunder. Garrick's past is shrouded in darkness, and his powers are both feared and respected by those who encounter him.\n\nName: Aria Swiftfoot\nBiography: Aria Swiftfoot is a nimble and agile rogue who roams the bustling city streets of Aranthia. Growing up as an orphan, she learned the art

---


## Sequential Story Scene Generation:


In [28]:
from langchain.memory import SimpleMemory

# Extracting the scenes using .split('\n') and removing empty strings:
scenes = [scene for scene in story_result["scenes"].split("\n") if scene]

character_script_prompt = PromptTemplate(
    template="""Given the following characters: {characters} and the genre: {genre}, create an effective character script for a scene.

    You must follow the following principles:
    - Use the Previous Scene Summary: {previous_scene_summary} to avoid repeating yourself.
    - Use the Plot: {plot} to create an effective scene character script.
    - Currently you are generating the character dialogue script for the following scene: {scene}

    ---
    Here is an example response:
    SCENE 1: ANNA'S APARTMENT

    (ANNA is sorting through old books when there is a knock at the door. She opens it to reveal JOHN.)
    ANNA: Can I help you, sir?
    JOHN: Perhaps, I think it's me who can help you. I heard you're researching time travel.
    (Anna looks intrigued but also cautious.)
    ANNA: That's right, but how do you know?
    JOHN: You could say... I'm a primary source.

    ---
    SCENE {index}:

    """,
    input_variables=[
        "characters",
        "genre",
        "plot",
        "scene",
        "previous_scene_summary",
        "index",
    ],
)

summarize_prompt = PromptTemplate(
    template="""Given a character script create a summary of the scene. Character script: {character_script}""",
    input_variables=["character_script"],
)

character_script_chain = LLMChain(
    llm=chat, prompt=character_script_prompt, output_key="character_script"
)

summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt, output_key="summary")
previous_scene_summary = ""

# Creating a simple memory to store the plot, characters, and genre:
memory = SimpleMemory(
    memories={
        "plot": story_result["plot"],
        "characters": story_result["characters"],
        "genre": "fantasy",
    }
)

generated_scenes = []

# You might want to use tqdm here to track the progress, or use all of the scenes
# for index, scene in enumerate(scenes):
for index, scene in enumerate(scenes)[0:5]:
    print(f"Generating scene {index + 1}...")
    scene_generation_chain = SequentialChain(
        chains=[character_script_chain, summarize_chain],
        memory=memory,
        input_variables=[
            "scene",
            "previous_scene_summary",
            "index",
        ],
        verbose=True,
        output_variables=["character_script", "summary"],
    )

    llm_result = scene_generation_chain(
        {
            "characters": story_result["characters"],
            "genre": "fantasy",
            "plot": story_result["plot"],
            "scene": scene,
            "previous_scene_summary": previous_scene_summary,
            "index": index + 1,
        }
    )

    # Updating the previous scene summary:
    previous_scene_summary = llm_result["summary"]

    # Store the generated scenes:
    generated_scenes.append(
        {"character_script": llm_result["character_script"], "scene": scenes[index]}
    )

Generating scene 1...


> Entering new SequentialChain chain...

> Finished chain.
Generating scene 2...


> Entering new SequentialChain chain...

> Finished chain.
Generating scene 3...


> Entering new SequentialChain chain...

> Finished chain.
Generating scene 4...


> Entering new SequentialChain chain...

> Finished chain.
Generating scene 5...


> Entering new SequentialChain chain...

> Finished chain.
Generating scene 6...


> Entering new SequentialChain chain...

> Finished chain.
Generating scene 7...


> Entering new SequentialChain chain...

> Finished chain.
Generating scene 8...


> Entering new SequentialChain chain...

> Finished chain.
Generating scene 9...


> Entering new SequentialChain chain...

> Finished chain.
Generating scene 10...


> Entering new SequentialChain chain...

> Finished chain.
Generating scene 11...


> Entering new SequentialChain chain...

> Finished chain.
Generating scene 12...


> Entering new SequentialChain chain...

> Finished chain.
G

KeyboardInterrupt: 

In [29]:
generated_scenes

[{'character_script': "EXT. ENCHANTED FOREST - DAY\n\n(The tranquil forest of Eldoria is bathed in golden sunlight, its beauty marred by an encroaching darkness. ELARA DAWNSHIELD, an elven archer with piercing green eyes and a determined expression, stands at the edge of the forest, her bow ready.)\n\nELARA: (whispering)\nSomething isn't right... The forest feels different today.\n\n(Suddenly, a strong gust of wind blows through the trees, carrying with it a sense of foreboding. Elara's eyes narrow as she sees the forest beginning to wither and decay.)\n\nELARA: (calling out)\nWise sages, what is happening to our home?\n\n(A voice echoes through the forest, originating from the ANCESTRAL TREE, the heart of Eldoria's magic.)\n\nANCESTRAL TREE: (whispering)\nElara Dawnshield, the darkness has descended upon us. Our very essence is under threat.\n\n(Elara's grip on her bow tightens, determination burning in her eyes.)\n\nELARA: I will not allow our home to be consumed by darkness. Tell me

In [30]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import pandas as pd

In [31]:
df = pd.DataFrame(generated_scenes)

In [32]:
all_character_script_text = "\n".join(df.character_script.tolist())

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=200
)

docs = text_splitter.create_documents([all_character_script_text])

In [33]:
chain = load_summarize_chain(llm=chat, chain_type="map_reduce")
# Remember to use .run() instead of .invoke() when using load_summarize_chain:
summary = chain.run(docs)
print(summary)

Elara, Garrick, and Aria, a group of warriors, join forces to protect their homeland, Eldoria, from an encroaching darkness. Guided by the Ancestral Tree, they journey to the Forgotten Caverns to uncover the source of the darkness. They face challenges, defeat mythical creatures, and eventually confront and defeat the sorcerer responsible for the darkness. They celebrate their victory but remain vigilant in protecting Eldoria.
